### beforehair.py 불러와서 사용자의 현재 얼굴형, 헤어스타일 분석 *--> while 문으로 변경*

##### db에 있는 사용자인지 확인해서 모델 돌리고 결과값 나오게끔 반복문/조건문 만들기

In [1]:
# 필요 패키지
import os
import json
import requests

import beforehair

#mqtt통신 프로그램 모듈 추가
import paho.mqtt.client as mqtt

# cpu만 강제 사용
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

#통신을 위한 인스턴스 생성, 서버 닫혀있을 시 에러 발생(클라우드에게 문의)
try:
    client = mqtt.Client()
    client.connect('54.150.133.192', 1883)
    print("mqtt connect success")
except:
    print("mqtt server error")

while True:
    base_path = './hair_image/before_hair/'
    storage_path = './hair_image/storage/'
    
    # before_hair 디렉토리 내의 파일 개수 확인
    dirListing = os.listdir(base_path)
    # print(dirListing)
    
    # 개수가 0이 아니면 try 구문 실행
    if len(dirListing) != 0:
        try:
            # 들어온 폴더 이름 추출
            img_test = os.listdir(base_path)[1] 
            
            # 폴더 이름에서 name, phone_number 추출 -> 이따가 permission 에러 해결하면 성별,컷/펌도 집어넣을 것!!!
            name = img_test.split('_',1)[0] 
            phone_number = img_test.split('_',1)[1]
            phone_number = phone_number.split('.',1)[0]

            # 사진 위치 경로 / 복제 시킬 경로
            base_path = './hair_image/before_hair/'
            storage_path = './hair_image/storage/'
            
            # 얼굴형, 현재 헤어스타일, 헤어 길이 반환
            face_shape, hair_style, hair_length = beforehair.face_classify(name,phone_number,base_path,storage_path)
            
            # 성별, 컷/펌 임의지정
            gender = 'man'
            style = 'cut'

            if hair_length == '장발':
                hair_length = '1'
            elif hair_length == '중발':
                hair_length = '2'
            elif hair_length == '단발':
                hair_length = '3'
            elif hair_length == '여숏':
                hair_length = '4'
            else:
                hair_length = '5'
            
            # 헤어스타일 추천
            path, hair = beforehair.hairstyle_recom(gender, face_shape, style, hair_length)

            # 사용자 정보 json 파일로 반환
            after_table = {'type':'bigdata',
                            'name':name,
                            'face_shape':face_shape,
                            'before_hair':hair_style,
                            'recom_hair1':path['1'],
                            'recom_hair2':path['2'],
                            'recom_hair3':path['3'],
                            'recom_hair4':path['4'],
                            'after_hair':'보브'}

            print(after_table)

            # 정보 전송
            headers = {'Content-Type': 'application/json; chearset=utf-8'}
            res = requests.post('http://was.t-seonghun.net:8000/insertdata/', data=json.dumps(after_table), headers=headers)
            print(res)
            
            # mqtt 통신으로 json 전달, 서버 닫혀있을 시 에러
            after_table2 = {'type':'bigdata',
                            'name':name,
                            'face_shape':face_shape,
                            'before_hair':hair_style,
                            'recom_hair1':path['1'],
                            'content1':'1번 스타일입니다',
                            'recom_hair2':path['2'],
                            'content2':'2번 스타일입니다',
                            'recom_hair3':path['3'],
                            'content3':'3번 스타일입니다',
                            'recom_hair4':path['4'],
                            'content4':'4번 스타일입니다',
                            'after_hair':'보브'}

            print(after_table2)
            
            try:
                client.publish('json', json.dumps(after_table2), 1)
            except:
                print("publish error")

        except:
            pass
        
        break


mqtt connect success
